<table>
<tr><td><img style="height: 150px;" src="images/geo_hydro1.jpg"></td>
<td bgcolor="#FFFFFF">
    <p style="font-size: xx-large; font-weight: 900; line-height: 100%">AG Dynamics of the Earth</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Jupyter notebooks</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Georg Kaufmann</p>
    </td>
</tr>
</table>

# Dynamic systems: 1. Introduction 
# Lid-driven cavity flow with openFoam
----
*Georg Kaufmann,
Geophysics Section,
Institute of Geological Sciences,
Freie Universität Berlin,
Germany*

In this notebook, we create and run the **lid-diven cavity problem** with the help of the `openFOAM` tutorial.

See also [tutorial](https://www.openfoam.com/documentation/tutorial-guide/2-incompressible-flow/2.1-lid-driven-cavity-flow)

----
## Typical problem

A typical problem solved with openFoam is the **lid-driven cavity** problem from fluid dynamics.

<div>
<img src="images/cavity.jpg" style="width:8cm">
<img src="images/example_cavity.jpg" style="width:8cm">
</div>

A box with dimensions $l_x$, $l_y$, and $l_z$ [m] is filled with a fluid of density $\rho$ [kg/m$^3$] and
kinematic viscosity $\nu$ [m$^2$/s].

The top surface to moved to the right with a constant velocity $u_{top}$ [m/s].

We want to calculate both 
- the **velocity** $u(x,y,z)$ [m/s]
- the **pressure** $p(x,y,z)$ [Pa]

in the model domain.

We have to solve both the continuity equation and the equation of motion. We define the fluid as **incompressible**.
We then have as differential equations:

**Continuity equation**
$$
\nabla \cdot u = 0
$$
**Equation of motion**
$$
\frac{\partial u}{\partial t}
+ u \cdot \nabla u
= - \nabla p
+ \nabla^2 u
$$
Here. all variables have been normalized:
- Time normalisation: $\frac{L^2}{\nu}$
- Length normalisation: $L$
- Velocity normalisation: $\frac{\nu}{L}$
- Pressure normalisation: $\frac{\rho \nu^2}{L^2}$

We will solve this example, which is part of the openFoam tutorial, with the `icoFoam` solver.

----
## Prepare solution

We open `openFoam` in the docker environment in a terminal (remember to use a `bash` shell):
~~~bash
$ docker run -it --rm -v $PWD/tmp:/home/tmp openfoam/openfoam9-graphical-apps
~~~

In the shell, we make a copy of the tutorial to our home directory:
~~~bash
$ cd /tmp
$ cp -r $FOAM_TUTORIALS/incompressible/icoFoam/cavity/cavity .
$ ls
cavity
$ cd cavity
$ ls -l
drwxr-xr-x 4 ignore ignore   128 Jan 12 09:34 0
drwxr-xr-x 3 ignore ignore    96 Jan 12 09:34 constant
drwxr-xr-x 6 ignore ignore   192 Jan 12 09:34 system
~~~

An `openFoam run` is structured as directory. 
A typical directory run looks like:

~~~
|-- cavity
    |-- 0
    |-- constant
    |-- system
~~~

----
### `0`

Here, the directory `0` contains the boundary conditions,
the directory `constant` parameter values, and the directory
`system` settings for the drivers and algorithms used.

Files contain initial conditions for variables, in this case
`p` and `U`. The first one pressure (divided by
density in the incompressible cases), the second one velocities.

Check the velocity file `U`: 

It contains the dimensions
of the field variable, a uniform values for velocity,
and the definition of side walls, with the `movingWall`
having a prescribed velocity of 1 m/s in x direction.

<details><summary><b>Show U</b></summary>
    
~~~
dimensions      [0 1 -1 0 0 0 0];
internalField   uniform (0 0 0);
boundaryField
{
    movingWall
    {
        type            fixedValue;
        value           uniform (1 0 0);
    }
    fixedWalls
    {
        type            noSlip;
    }
    frontAndBack
    {
        type            empty;
    }
}
~~~
    
</details>

The pressure file `p` has a similar structure, but no-flow boundary
conditions:

<details><summary><b>Show p</b></summary>
    
~~~
dimensions      [0 2 -2 0 0 0 0];
internalField   uniform 0;
boundaryField
{
    movingWall
    {
        type            zeroGradient;
    }
    fixedWalls
    {
        type            zeroGradient;
    }
    frontAndBack
    {
        type            empty;
    }
}
~~~
    
</details>

The used `dimensions` array keeps track of the SI units, the
order is: `kg m s K mol cd A`

----
### `constant`

The only file is `transportProperties`, holding material parameters.
Here, the **kinematic viscosity** $\nu$ is defined as $\nu=0.01$ m$^2$/s.

<details><summary><b>Show transportProperties</b></summary>
    
~~~
nu              [0 2 -1 0 0 0 0] 0.01;
~~~
    
</details>

----
### `system`

Geometry is defined in `blockMeshDict`.
Note the re-scaling of length, with `convertToMeters 0.1` all length
are rescaled by 1/10. (We should avoid this ...).

<details><summary><b>Show blockMeshDict</b></summary>
    
~~~
convertToMeters 0.1;

vertices
(
    (0 0 0)
    (1 0 0)
    (1 1 0)
    (0 1 0)
    (0 0 0.1)
    (1 0 0.1)
    (1 1 0.1)
    (0 1 0.1)
);

blocks
(
    hex (0 1 2 3 4 5 6 7) (20 20 1) simpleGrading (1 1 1)
);
edges
(
);

boundary
(
    movingWall
    {
        type wall;
        faces
        (
            (3 7 6 2)
        );
    }
    fixedWalls
    {
        type wall;
        faces
        (
            (0 4 7 3)
            (2 6 5 1)
            (1 5 4 0)
        );
    }
    frontAndBack
    {
        type empty;
        faces
        (
            (0 3 2 1)
            (4 5 6 7)
        );
    }
);

mergePatchPairs
(
);
~~~

</details>

The file `controlDict` one holds most of the parameters for the run, e.g. time step,
step size, output options, ...

<details><summary><b>Show controlDict</b></summary>
    (not here ...)
</details>

The file `fvSchemes` defines the discretisation methods for solving.

<details><summary><b>Show fvSchemes</b></summary>
    (not here ...)
</details>

The file `fvSolution` defines the solvers and their tolerances.

<details><summary><b>Show fvSolution</b></summary>
    (not here ...)
</details>

----
## Run example

We start to process the example by first **cleaning up** and then
**creating the mesh**:

~~~bash
$ foamCleanTutorials
$ blockMesh
~~~

In `constant/polyMesh`, several files have been created.
To check the mesh, use `Paraview` and navigate to the folder into
`system`, open the files `controlDict`, load with openFoam reader, 
and activate!

We then run openfoam with

~~~bash
$ icoFoam
~~~

The run created several output directories, one for each pre-defined
time steps (0.1, 0.2, ...).

~~~
|-- cavity
    |-- 0
    |-- 0.1
    |-- 0.2
    |-- 0.3
    |-- 0.4
    |-- 0.5
    |-- constant
    |-- system
~~~

----
## Postprocessing with `ParaView`

We want to view the results. There are two options, depending on your version of `ParaView`:
- **Create a file:**

~~~bash
$ touch cavity.foam
~~~

This make it easy to use `paraView` now:

- Open `paraView`, then use open dialog and navigate to the **cavity** example.
The file `cavity.foam` should be visible. Double click to open and load data.

- **Run a post-processing tool:**

~~~bash
$ foamToVTK
~~~

### Make plot

- Show $p$ field and colorbars ...

- Plot $p$ contours ...

- Plot velocity arrows for $U$ ...

- Plot streamlines (Stream tracer) ...

- Make streamlines nicer (Tubes) ...


... done